# This is just a notebook to visualise 1kHz filtered raw data

## Setup everything

### Import packages

In [1]:
import os
import numpy as np
import matplotlib

from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer, InMemoryAnalogSignalSource
from ephyviewer import AnalogSignalSourceWithScatter

# add the Contrib dir that contains all tools developped by MB : mbTools.py
#sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'python'))
#print(os.path.join(os.path.dirname(sys.path[0]),'python'))
from mbTools import mbTools

### Import local config, create it if inexistant
All user-specific configuration (projects, defautl path to files...) are stored in a file named localConfig.ini in the python subfolder of AudreyHayLab repo. It is ignored by git so that it remains truely local. If the file does not exist at beginning, it is created with default values that can be modified at whishes.

In [2]:
config = mbTools.localConf()
rawDataPath = config['DATA']['path']
print(f'All raw data are expected to be found in the folder: {rawDataPath}')
analysisPath = config['ANALYSIS']['path']
print(f'All analysis will be saved in the folder: {analysisPath}')
config.printAll()

Local config file loaded from localConfig.ini
All raw data are expected to be found in the folder: /Users/mb
All analysis will be saved in the folder: /Users/mb/Analysis
DATA
[('path', '/Users/mb')]
ANALYSIS
[('path', '/Users/mb/Analysis'),
 ('projecttype', '0'),
 ('animalid', '0'),
 ('projectid', 'AProject'),
 ('subprojectid', 'OneOfItsSubProject'),
 ('conditionid', 'control'),
 ('recordingid', '0')]
AProject.OneOfItsSubProject
[('design', '0'),
 ('nanimal', '6'),
 ('conditions', "['control']"),
 ('nrecordings', '1')]
Annie.NPX
[('design', '0'),
 ('nanimal', '6'),
 ('conditions', "['control']"),
 ('nrecordings', '1')]


## Choose experiment
Select the experiment to display. If the experiment was already analyzed, a saved_dictionary.pkl was created and contains all necessary variables. Select this file. Otherwise select the raw data recording file.
>**If you have a file with channel mapping somewhere**, we should make sure it is properly translated into a dict.pkl

In [3]:
currentFile = None
%store -r currentFile
print(currentFile)
try:
    theExpe = mbTools.expeConfigDict(currentFile)
except Exception as error:
    print(error)
    theExpe = mbTools.expeConfigDict()

/Users/mb/Analysis/AProject/OneOfItsSubProject/control/0/saved_dictionary.pkl
Local config file loaded from localConfig.ini
the file is /Users/mb/Analysis/AProject/OneOfItsSubProject/control/0/saved_dictionary.pkl


FileChooser(path='/Users/mb/Analysis/AProject/OneOfItsSubProject/control/0', filename='saved_dictionary.pkl', …

### Possibility to change raw data path 
if for some reason the path to the raw data is wrong, you can update it here

In [4]:
theExpe.rawDataSelector()

FileChooser(path='/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/python', filename='', title='<b>ePhys data</b>…

## Load Data

### Map the whole data into memory

In [5]:
if False:
    dpath = None #  '//10.69.168.1/crnldata/waking/audrey_hay/NPX/NPX4_claustrum/Expe_2024-07-18_12-00-43/'
    # %store dpath
    %store -r dpath
    print(dpath)
    theExpe.rawDataPath = dpath
thedata = mbTools.experiment(theExpe)

FileChooser(path='/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/recording8/continuous/Rhythm_FPGA-133.0', file…

In [6]:
Timestamps = thedata.loadRecording_TimeStamps()
print(Timestamps)

/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/python
None


In [7]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
thedata.analyseExpe_findData()

/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/recording8/continuous/Rhythm_FPGA-133.0
found some continuous.dat files
carrefull, to match my case, numChannels is set to 64
data not recorded with Bonsai
importing /Users/mb/Documents/Syntuitio/AudreyHay/PlanB/recording8/continuous/Rhythm_FPGA-133.0/continuous.dat
OE32channels.bin file loaded, with 64 channels and 23513856 datapoint


### Extract submatrix of interest

In [8]:
#initiate combined and channelLabels
combined =  {}
channelLabels = {}
sample_rates = {}

In [10]:
#LFP
if 'OE_LFP' in thedata.data:
    combined['LFP'] = thedata.data['OE_LFP'].combineStructures(None)[:,:16]
    channelLabels['LFP'] = thedata.data['OE_LFP'].channelLabels[:16]
    sample_rates['LFP'] = 1000
    print("LFP data combined")
else:
    print("no LFP data to combine")

LFP data combined


In [ ]:
#NPX
if 'NPX' in thedata.data:
    combined['NPX'] = thedata.data['NPX'].signal['spike']
    channelLabels['NPX'] = thedata.data['NPX'].channelLabels
    sample_rates['NPX'] = 30000
    print("NPX data combined")
else:
    print("no NPX data to combine")

In [11]:
%gui qt
app = mkQApp()

t_start = 0.

TTL = Timestamps

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: TTL, 1: TTL}
#and asign them to some channels each
scatter_channels = {0: [0, 12], 1: [0, 1]}
#source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True)

if 'LFP' in combined:
    source = InMemoryAnalogSignalSource(combined['LFP'], sample_rates['LFP'], t_start, channel_names=channelLabels['LFP'])
    view1 = TraceViewer(source=source, name = 'LFP')
    #view1 = TraceViewer(source=source)

    #Parameters can be set in script
    view1.params['display_labels'] = True
    view1.params['scale_mode'] = 'same_for_all'
    view1.auto_scale()

    cmap = matplotlib.colormaps["hsv"]#Wistia"]
    nCh = len(view1.by_channel_params.children())
    for ch in range(nCh):
        #view1.by_channel_params[f'ch{ch}', 'gain'] = 0.00002
        #view1.by_channel_params[f'ch{ch}', 'offset'] = 0.1
        view1.by_channel_params[f'ch{ch}', 'color'] = matplotlib.colors.to_hex(cmap(ch/nCh), keep_alpha=False)
        pass

    #create a time freq viewer conencted to the same source
    view2 = TimeFreqViewer(source=source, name='tfr')
    #view2.params['show_axis'] = False
    #view2.params['timefreq', 'deltafreq'] = 1
    #view2.by_channel_params['ch3', 'visible'] = True

    win.add_view(view1)
    win.add_view(view2)


if 'NPX' in combined:
    view3 = TraceViewer.from_numpy(combined['NPX'], sample_rates['NPX'], t_start, 'NPX', channel_names=channelLabels['NPX'])
    #view1 = TraceViewer(source=source)
    win.add_view(view3)

    #Parameters can be set in script
    view3.params['display_labels'] = True
    view3.params['scale_mode'] = 'same_for_all'
    view3.auto_scale()

    cmap = matplotlib.colormaps["hsv"]#Wistia"]
    nCh = len(view3.by_channel_params.children())
    for ch in range(nCh):
        #view3.by_channel_params[f'ch{ch}', 'gain'] = 0.00002
        #view3.by_channel_params[f'ch{ch}', 'offset'] = 0.1
        view3.by_channel_params[f'ch{ch}', 'color'] = matplotlib.colors.to_hex(cmap(ch/nCh), keep_alpha=False)
        pass

#Run
win.show()
#app.exec()  #if commented, the app is shown and fonctionnal. Maybe detecting buttons. the Python icon doesn't close any better

debug True
QT_MODE PySide6


: 